In [100]:
import numpy as np
import skimage
import utils
import pathlib

In [101]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block)

    L = 256
    # Compute normalized histogram
    norm_hist = np.zeros(L)
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            norm_hist[im[i][j]] += 1
    mn = im.shape[0] * im.shape[1]
    norm_hist = norm_hist * (1/mn)
    #assert norm_hist.sum() == 1.0

    #Computing cumulative sum, contains prob that a pixel is assigned to a value in the given threshold
    cum_sums = np.zeros(L)
    for k in range(L):
        cum_sums[k] = sum(norm_hist[i] for i in range(k))

    #Compute the cumulative means
    cum_means = np.zeros(L)
    for k in range(L):
        cum_means[k] = sum(norm_hist[i]*i for i in range(k))

    #Global intensity mean
    glob_int_mean = sum(norm_hist[i]*i for i in range(L))
    #glob_int_mean = cum_means[L-1] #this should actually hold, but it does not sometimes for some weird reason

    #Between class variance and largest k
    bet_class_var = np.zeros(L)
    for k in range(L):
        numerator = (glob_int_mean * cum_sums[k] - cum_means[k])**2
        denominator = (cum_sums[k]*(1-cum_sums[k])) #NB! this term is 0 sometimes, resulting in some nan values, especially for the earliest between class variances
        bet_class_var[k] = numerator/denominator

    #Otsu threshold, NB! does not consider multiple ks
    largest_k = np.nanargmax(bet_class_var)

    threshold = largest_k

    return threshold
    ### END YOUR CODE HERE ###

In [102]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png"),
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images\thumbprint.png
Found optimal threshold: 154
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\rice-shaded.png
Found optimal threshold: 135
Saving image to: image_processed\rice-shaded-segmented.png


C:\Users\willi\AppData\Local\Temp\ipykernel_21196\418176960.py:43: RuntimeWarning: invalid value encountered in scalar divide
  bet_class_var[k] = numerator/denominator
